# A real machine learning pipeline

In this notebook we will demonstrate how scikit-learn can be used to assemble a complete machine learning pipeline to predict house prices on the Boston housing dataset. By using the [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)-class we can stitch together pre-processing and model fitting in a very convenient way. 

Using the pipelining we also avoid data leakage. Probably the most common reason for data leakage is that preprocessing steps are accidentally fitted on both train- and test-data. If we for instance use standard scaling as preprocessing, such mistake would mean that the variables are scaled using parameters calculated partly on the test-set. This means that the model will get access to "knowledge" leaked from the test-set into the training, which may give a overly optimistic evaluation.

We don't want optimistic evaluations, we want honest ones so we can report honest results back to our managers.

**Feel free to explore other regression models in your own cells before**

The scikit-learn user guide has plenty of information on different [regression](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning) and [preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing) methods.

In [1]:
import pickle

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.linear_model import LinearRegression, RidgeCV

First we read our data and split out response to separate series.

In [2]:
df = pd.read_csv('boston-housing.csv')

response = df['target']
df.drop('target', axis=1, inplace=True)

We split out 30 \% of the data to use for validating our model.

We are not aware of any dependencies in our data, so we make the split randomly

In [3]:
np.random.seed(1337)

x_train, x_test, y_train, y_test = train_test_split(df, response, test_size=.3)

Now we define a convenience-function that we will use to cross-validate our pipelines

In [4]:
def crossvalidated_mean_squared_error(regressor, x, y):
    scoring_fn = make_scorer(mean_squared_error)
    fold_scores = cross_val_score(regressor, x, y, cv=5, scoring=scoring_fn)
    return fold_scores.mean()

# 1st attempt - Linear regression

We start simple, with a linear regression using all variables.

In [5]:
lr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LinearRegression())
])

mse_cv = crossvalidated_mean_squared_error(lr_pipeline, x_train, y_train)

print(f'Mean Square Error averaged accross cross-validation: {mse_cv}')

Mean Square Error averaged accross cross-validation: 20.3953453272921


# 2nd attempt - Polynomial regression

To see if we can improve our results, we may include polynomial terms as well (square terms and interactions).

In [6]:
poly_lr_pipeline = Pipeline([
    ('polynomial', PolynomialFeatures()),
    ('scaler', StandardScaler()),
    ('model', LinearRegression())
])

mse_cv = crossvalidated_mean_squared_error(poly_lr_pipeline, x_train, y_train)

print(f'Mean Square Error averaged accross cross-validation: {mse_cv}')

Mean Square Error averaged accross cross-validation: 41.239820413420496


Whoops. Error is twice as large using simple regression with polynomial features.

# 3rd attempt - Polynomial features with Ridge regression

Our naive attempt on polynomial features was probably due to overfitting. We try to use Ridge regression, which is a popular method to combat overfitting.

In [7]:
poly_ridge_pipeline = Pipeline([
    ('polynomial', PolynomialFeatures()),
    ('scaler', StandardScaler()),
    ('model', RidgeCV(cv=3))
])

mse_cv = crossvalidated_mean_squared_error(poly_ridge_pipeline, x_train, y_train)

print(f'Mean Square Error averaged accross cross-validation: {mse_cv}')

Mean Square Error averaged accross cross-validation: 11.30858850179826


C:\Users\RickardS\Anaconda3\envs\umejug-env\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\RickardS\Anaconda3\envs\umejug-env\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\RickardS\Anaconda3\envs\umejug-env\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


# Results

According to cross-validation, our best model is to use polynomial features combined with ridge regression.

We therefore fit our final model on all training data and report test-set performance.

In [8]:
poly_ridge_pipeline.fit(x_train, y_train)

test_mean_squared_error = mean_squared_error(y_test, poly_ridge_pipeline.predict(x_test))

print(f'Test-set Mean Square Error: {test_mean_squared_error}')

Test-set Mean Square Error: 20.138701207866074


We see that the test-set error is quite a bit higher than the validation error. This is common, and it is the most honest estimate you get.

**Don't look at test-set error before deciding which model to use!**
You want to know roughly how well the model will work in the future. The model is supposed to be the one you actually think works best. NOT the one that just happen to perform best on this particular train-test split.

Lastly, we persist our model on disk so we can use it in the future.

In [9]:
with open('boston-housing_polynomial-ridge.pickle', 'wb') as f:
    pickle.dump(poly_ridge_pipeline, f)

# Your attempts

Follow the structure of the examples above and try to implement your own pipeline.